In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = (SparkSession.builder.master("local")
             .appName("Bronze to Silver")
             .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.1,com.amazonaws:aws-java-sdk-bundle:1.11.375")
             .config("spark.hadoop.fs.s3a.access.key", "minio_access_key")
             .config("spark.hadoop.fs.s3a.secret.key", "minio_secret_key")
             .config("spark.hadoop.fs.s3a.path.style.access", "true")
             .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
             .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
             .getOrCreate())


24/06/01 23:19:44 WARN Utils: Your hostname, datdt resolves to a loopback address: 127.0.1.1; using 192.168.2.12 instead (on interface wlp0s20f3)
24/06/01 23:19:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/datdt/anaconda3/envs/de/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/datdt/.ivy2/cache
The jars for the packages stored in: /home/datdt/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ae14e016-4a6c-4d27-a235-a4abe5877499;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.901 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 147ms :: artifacts dl 6ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.1 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.11.375 by [com.amazonaws#aws-java-sdk-bundle;1.11.901] in [default]
	------------------------------------------------------------------

24/06/01 23:19:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df = spark.read.parquet(f"s3a://bronze-data/raw_data")
df.show()

24/06/01 23:19:47 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
+---+-------------+--------+---------+-------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+--------+
|age|          job| marital|education|default|balance|housing|loan|  contact|day|month|duration|campaign|pdays|previous|poutcome|
+---+-------------+--------+---------+-------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+--------+
| 19|self-employed|divorced|  unknown|    yes|  -2820|    yes| yes| cellular| 31|  dec|    1933|       4|   15|       3| failure|
| 79| entrepreneur|  single|  primary|    yes|  -5277|    yes| yes|  unknown| 23|  aug|    1919|       8|    3|       7|   other|
| 71| entrepreneur| married|secondary|    yes|  -6381|    yes|  no|  unknown| 10|  sep|     332|       9|   10|       7| failure|
| 78|      student|  single|secondary|    yes|   9437|    yes|  no|telephone|

In [11]:
# Transformations with categorical data
from pyspark.sql.functions import when

df = df.withColumn('default', when(df['default'] == 'yes', 1).otherwise(0))
df = df.withColumn('housing', when(df['housing'] == 'yes', 1).otherwise(0))
df = df.withColumn('loan', when(df['loan'] == 'yes', 1).otherwise(0))
df.show()


+---+-------------+--------+---------+-------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+--------+----+
|age|          job| marital|education|default|balance|housing|loan|  contact|day|month|duration|campaign|pdays|previous|poutcome|load|
+---+-------------+--------+---------+-------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+--------+----+
| 19|self-employed|divorced|  unknown|      0|  -2820|      0|   1| cellular| 31|  dec|    1933|       4|   15|       3| failure|   0|
| 79| entrepreneur|  single|  primary|      0|  -5277|      0|   1|  unknown| 23|  aug|    1919|       8|    3|       7|   other|   0|
| 71| entrepreneur| married|secondary|      0|  -6381|      0|   0|  unknown| 10|  sep|     332|       9|   10|       7| failure|   0|
| 78|      student|  single|secondary|      0|   9437|      0|   0|telephone|  9|  mar|    2194|       5|    3|      10| unknown|   0|
| 52|       admin.|  single| tertiary|      0|   8987| 

In [26]:
from pyspark.sql.functions import when

# Replace 'divorced' with '0', 'single' with '1', and 'married' with '2'
df = df.withColumn('marital', when(df['marital'] == 'divorced', 0)
                                 .when(df['marital'] == 'single', 1)
                                 .when(df['marital'] == 'married', 2)
                                 .otherwise(df['marital']))

df = df.withColumn('education', when(df['education'] == 'unknown', 0).
                                    when(df['education'] == 'primary', 1).
                                    when(df['education'] == 'secondary', 2).
                                    when(df['education'] == 'tertiary', 3).
                                    otherwise(df['education']))

df = df.withColumn('job', when(df['job'] == 'admin.', 0)
                          .when(df['job'] == 'unknown', 1)
                          .when(df['job'] == 'unemployed', 2)
                          .when(df['job'] == 'management', 3)
                          .when(df['job'] == 'housemaid', 4)
                          .when(df['job'] == 'entrepreneur', 5)
                          .when(df['job'] == 'student', 6)
                          .when(df['job'] == 'blue-collar', 7)
                          .when(df['job'] == 'self-employed', 8)
                          .when(df['job'] == 'retired', 9)
                          .when(df['job'] == 'technician', 10)
                          .when(df['job'] == 'services', 11)
                          .otherwise(df['job']))

df = df.withColumn('contact', when(df['contact'] == 'unknown', 0)
                               .when(df['contact'] == 'telephone', 1)
                               .when(df['contact'] == 'cellular', 2)
                               .otherwise(df['contact']))

df = df.withColumn('poutcome', when(df['poutcome'] == 'unknown', 0)
                                .when(df['poutcome'] == 'other', 1)
                                .when(df['poutcome'] == 'failure', 2)
                                .when(df['poutcome'] == 'success', 3)
                                .otherwise(df['poutcome']))
df.show()

+---+---+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+----+---------------+
|age|job|marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|load|marital_encoded|
+---+---+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+----+---------------+
| 19|  8|      0|        0|      0|  -2820|      0|   1|      2| 31|  dec|    1933|       4|   15|       3|       2|   0|            1.0|
| 79|  5|      1|        1|      0|  -5277|      0|   1|      0| 23|  aug|    1919|       8|    3|       7|       1|   0|            0.0|
| 71|  5|      2|        2|      0|  -6381|      0|   0|      0| 10|  sep|     332|       9|   10|       7|       2|   0|            2.0|
| 78|  6|      1|        2|      0|   9437|      0|   0|      1|  9|  mar|    2194|       5|    3|      10|       0|   0|            0.0|
| 52|  0|      1|        3|      0

In [25]:
df.groupBy('job').count().show()

+-------------+-----+
|          job|count|
+-------------+-----+
|   management|  158|
|      retired|  144|
|      unknown|  190|
|self-employed|  160|
|      student|  177|
|  blue-collar|  178|
| entrepreneur|  163|
|       admin.|  155|
|   technician|  167|
|     services|  166|
|    housemaid|  170|
|   unemployed|  172|
+-------------+-----+

